# Trial 3
Not sure if this will work either! :')

In [1]:
# Importing necessary libraries
import pandas as pd
import pickle

In [2]:
# Reading the data
with open('../df.pkl', 'rb') as f:
    df = pickle.load(f)

## Approach

In [3]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from tqdm import tqdm
from datetime import datetime, timedelta
import numpy as np

from datasketch import MinHash, MinHashLSH

/workspaces/storion_aggregation_analysis/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Downloading the necessary nltk data
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [6]:
# Pre-process the text data
# - Tokenize
# - Remove stop words
# - Lemmatize
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    words = word_tokenize(text)
    words = [word for word in words if word.isalnum()]
    words = [word for word in words if word not in stop_words]
    words = [lemmatizer.lemmatize(word) for word in words]

    return ' '.join(words)


pp_headlines = []
pp_content = []

for headline, content in tqdm(zip(df['title'], df['content']), total=len(df), desc='Pre-processing text'):
    pp_headlines.append(preprocess_text(headline))
    pp_content.append(preprocess_text(content))

Pre-processing text: 100%|██████████| 2977/2977 [00:18<00:00, 160.80it/s]


In [7]:
# class LSH:
#     def __init__(self, num_bands=8):
#         """Initializes the LSH object.

#         Args:
#             num_bands (int): Number of bands to divide the hash into.
#         """
#         self.num_bands = num_bands
#         self.buckets = [defaultdict(list) for _ in range(num_bands)]

#     def _get_band_hashes(self, simhash_value, band_size):
#         """Splits the simhash value into band hashes."""
#         for i in range(self.num_bands):
#             yield (simhash_value >> (i * band_size)) & ((1 << band_size) - 1)

#     def index(self, article_id, simhash_value):
#         """Indexes the text into LSH buckets based on its SimHash."""
#         hash_size = 64  # Default hash size for SimHash
#         band_size = hash_size // self.num_bands

#         for i, band_hash in enumerate(self._get_band_hashes(simhash_value, band_size)):
#             self.buckets[i][band_hash].append((article_id, simhash_value))

#     def query(self, simhash_value):
#         """Finds all articles that are similar to the given SimHash."""
#         hash_size = 64
#         band_size = hash_size // self.num_bands
#         candidates = set()

#         for i, band_hash in enumerate(self._get_band_hashes(simhash_value, band_size)):
#             if band_hash in self.buckets[i]:
#                 for item in self.buckets[i][band_hash]:
#                     candidates.add(item)

#         return candidates

In [8]:
# class SafeSimhash(Simhash):
#     def build_by_features(self, features):
#         """Override to handle cases where features might cause integer overflows."""
#         hasher = self.hashfunc
#         v = [0] * self.f
#         masks = [1 << i for i in range(self.f)]

#         for f, w in features.items():
#             h = int.from_bytes(hasher(f.encode('utf-8')), byteorder='big')
#             for i in range(self.f):
#                 v[i] += w if h & masks[i] else -w

#         self.value = 0
#         for i in range(self.f):
#             if v[i] > 0:
#                 self.value |= masks[i]

#         return self

In [9]:
class ArticleGroup:
    def __init__(self, article_minhash, initial_article):
        self.articles = [initial_article]
        self.dates = [initial_article['publication_date']]
        self.lsh = MinHashLSH(threshold=0.7, num_perm=128)
        self.lsh.insert(initial_article['index'], article_minhash)
        
    def get_temporal_bounds(self, k=1.5):
        """Calculate dynamic date boundaries using IQR"""
        if len(self.dates) == 1:
            return (self.dates[0] - timedelta(hours=24), 
                    self.dates[0] + timedelta(hours=24))
            
        timestamps = [d.timestamp() for d in self.dates]
        q1, q3 = np.percentile(timestamps, [25, 75])
        iqr = q3 - q1
        lower = datetime.fromtimestamp(q1 - k*iqr)
        upper = datetime.fromtimestamp(q3 + k*iqr)
        return lower, upper

    def is_within_temporal_bounds(self, new_date, k=1.5):
        lower, upper = self.get_temporal_bounds(k)
        return lower <= new_date <= upper

    def add_article(self, minhash, article):
        self.articles.append(article)
        self.dates.append(article['publication_date'])
        self.lsh.insert(article['index'], minhash)

    
class LSH:
    def __init__(self):
        self.groups = []
        self.global_lsh = MinHashLSH(threshold=0.7, num_perm=128)
        
    def process_article(self, minhash, article, k=1.5):
        # Find candidate matches using LSH
        candidate_ids = self.global_lsh.query(minhash)
        candidate_groups = [
            g for g in self.groups 
            if any(a['index'] in candidate_ids for a in g.articles)
        ]
        
        # Check temporal constraints
        # valid_groups = []
        # for group in candidate_groups:
        #     if group.is_within_temporal_bounds(article['publication_date'], k):
        #         valid_groups.append(group)
        valid_groups = candidate_groups

        # Create new group or merge with existing
        if not valid_groups:
            new_group = ArticleGroup(minhash, article)
            self.groups.append(new_group)
            self.global_lsh.insert(article['index'], minhash)
            return new_group
            
        # Add to first valid group (could be enhanced with best match selection)
        valid_groups[0].add_article(minhash, article)
        return valid_groups[0]

In [10]:
def generate_minhash(tokens, num_perm=128):
    m = MinHash(num_perm=num_perm)
    for token in tokens:
        m.update(token.encode('utf-8'))
    return m

In [11]:
# Compute SimHashes
content_lsh = LSH()
article_group_list = {}
for (idx, article), content_tokens in tqdm(zip(df.iterrows(), pp_content), desc='Computing MinHashes', total=len(pp_content)):
    article['index'] = idx
    minhash = generate_minhash(content_tokens)
    article_group_list[article['index']] = {
        'minhash': minhash,
        'group': content_lsh.process_article(minhash, article)
    }

Computing MinHashes:   0%|          | 11/2977 [00:00<01:28, 33.42it/s]

Computing MinHashes: 100%|██████████| 2977/2977 [01:03<00:00, 46.78it/s] 


In [12]:
# # LSH Group querying
# def hamming_distance(hash1, hash2):
#     """Computes the Hamming distance between two hashes."""
#     return bin(hash1 ^ hash2).count('1')

# def get_similar_articles_lsh(simhash_value):
#     similar_articles = content_lsh.query(simhash_value)
#     return similar_articles

In [13]:
# Let's test with case 1
case_1 = df.loc[df['event_id'] == 'case_1'].iloc[0]
case_1

title               Pennsylvania judge allows Elon Musk's PAC to c...
author              [Andrea Margolis, Fox News, Andrea Margolis Is...
publication_date                                                  NaT
source                                                       FOX News
url                 https://www.foxnews.com/politics/pennsylvania-...
summary                                                           NaN
content             A Pennsylvania judge is allowing Elon Musk's A...
tags                                                              NaN
images              [https://a57.foxnews.com/static.foxnews.com/fo...
event_id                                                       case_1
Name: 75e5337b-5962-45eb-9acd-45788497bf4d, dtype: object

In [14]:
# case_1_simhash = get_simhash_value(case_1['content'])
# print(f"SimHash value for case 1: {case_1_simhash}")
# similar_articles = get_similar_articles_lsh(case_1_simhash)

# print(f"Found {len(similar_articles)} similar articles")
# print("Similar articles:")
# for sim_article_idx, sim_article_hash in similar_articles:
#     print(f"- {hamming_distance(case_1_simhash, sim_article_hash)} {df.loc[sim_article_idx]['title']}")

In [15]:
len(content_lsh.groups)

12

In [16]:
content_lsh.groups[8].articles

[title               Why has Elon Musk been pushing for a governmen...
 author                                                            NaN
 publication_date                                                  NaT
 source                                                            BBC
 url                        https://www.bbc.co.uk/sounds/play/p0kd5691
 summary                                                           NaN
 content             Why has Elon Musk been pushing for a governmen...
 tags                                                              NaN
 images              [https://a1.api.bbc.co.uk/hit.xiti?&col=1&from...
 event_id                                                         none
 index                            222d3229-5315-4a93-969b-701dceb35b40
 Name: 222d3229-5315-4a93-969b-701dceb35b40, dtype: object,
 title               WATCH:  Husky doesn't understand why grandma d...
 author                                                     [Abc News]
 publication_date